## Project Mockup

In [1]:
struct S { double val = 1.0; };

In [2]:
%%python

python_vec = cppyy.gbl.std.vector(cppyy.gbl.S)(5)

In [3]:
%%python

print(python_vec[3].val)

1.0


In [4]:
%%python

class Derived(cppyy.gbl.S):
  def __init__(self):
    val = 0
res = Derived()
print(res)

<string>:1: RuntimeWarning: class "S" has no virtual destructor


In [5]:
__global__ void arr_sum(int n, double *x, double *sum) {
    for(int i = 0; i < n; i++)
        *sum +=x[i];
}

In [6]:
int n = 5;
double h_sum;
double *x = new double[n];

In [7]:
void setData(const std::vector<S>& a) {
    int i = 0;
    for(auto &s : a) {
        x[i] = s.val;
        i++;
    }
}

In [8]:
%%python

data_list = [1.0, 2.0, 3.0, 4.0, 5.0]
for c, i in enumerate(data_list):
    python_vec[c].val = i


In [9]:
%%python

cppyy.gbl.setData(python_vec)


In [10]:
double *d_x, *d_sum;
cudaMalloc((void **)&d_x, n * sizeof(double));
cudaMalloc((void **)&d_sum, sizeof(double));

In [11]:
cudaMemcpy(d_x, x, n * sizeof(double), cudaMemcpyHostToDevice);
cudaMemcpy(d_sum, &h_sum, sizeof(double), cudaMemcpyHostToDevice);

In [12]:
arr_sum<<<1, 1>>>(n, d_x, d_sum);

In [13]:
cudaMemcpy(&h_sum, d_sum, sizeof(double), cudaMemcpyDeviceToHost);

In [14]:
std::cout << "Sum: " << h_sum << std::endl;

Sum: 15


In [15]:
delete[] x;
cudaFree(d_x);
cudaFree(d_sum);